In [226]:
import os
from dotenv import load_dotenv
load_dotenv()

open_api_key = os.getenv("OPENAI_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")


In [227]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key=open_api_key,model = "gpt-3.5-turbo")


In [228]:
#creating supabase client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [229]:
from langchain_core.messages import HumanMessage
response = model.invoke([HumanMessage(content="Hi, I am Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS.")])

print(response)

content="Hello Sanatan! It's great to hear about your specialization in AI/ML and major in CS. How has your experience been in these fields so far?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 41, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7a4434a4-0dfa-4b53-a0ad-a8a0a8e1f112-0' usage_metadata={'input_tokens': 41, 'output_tokens': 33, 'total_tokens': 74, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [230]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, I am Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS."),
        AIMessage(content="Hello Sanatan! It's great to hear about your academic pursuits in AI/ML and Computer Science. How can I assist you today?"),
        HumanMessage(content="I am looking for an internship Can you help me with that?"),
    ]
)

AIMessage(content='Of course! I can provide you with some tips on how to secure an internship in your field. Here are a few steps you can take:\n\n1. Update your resume: Make sure your resume highlights your relevant skills, experience, and coursework in AI/ML and Computer Science.\n\n2. Build a strong portfolio: Showcase any projects, research, or coding samples that demonstrate your abilities in AI/ML.\n\n3. Network: Reach out to professionals in your field through LinkedIn, networking events, and career fairs. They may be able to provide you with leads on internship opportunities.\n\n4. Apply to internship programs: Check job boards, company websites, and internship platforms for openings in AI/ML and Computer Science.\n\n5. Prepare for interviews: Practice common interview questions and be ready to discuss your technical skills and experiences.\n\nGood luck with your internship search! If you need any more assistance, feel free to ask.', additional_kwargs={'refusal': None}, respons

In [231]:
#database function 
from supabase import create_client
def save_message_to_supabase(session_id, role, message):
    """Stores chat messages in Supabase and prints response for debugging."""
    try:
        data = {
            "session_id": session_id,
            "role": role,
            "message": message,
        }
        response = supabase.table("history").insert(data).execute()

        # ✅ Debugging Output
        print("Supabase API Response:", response)  
        return response
    except Exception as e:
        print("Supabase Error:", e)
        return None


def get_chat_history(session_id):
    """Retrieves chat history from Supabase."""
    response = supabase.table("history").select("*").eq("session_id", session_id).order("timestamp", desc=False).execute()
    return response.data

In [232]:
#now creating history rememberance system for the chatbot (Message history)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage


session_data = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    """Retrieves or initializes chat history for a session."""
    if session_id not in session_data:
        session_data[session_id] = {
            "history": ChatMessageHistory(),  # ✅ Initializes new history
            "language": "English"  # ✅ Default response language if not set
        }
    return session_data[session_id]["history"]

def set_session_language(session_id: str, language: str = "English"):
    """Stores the preferred response language for a given session."""
    if session_id not in session_data:
        session_data[session_id] = {
            "history": ChatMessageHistory(),
            "language": language
        }
    else:
        # ✅ Only update language if explicitly provided
        session_data[session_id]["language"] = language


In [233]:
def invoke_with_language(session_id: str, messages, language=None):
    """Handles chatbot invocation while ensuring memory, history & language persistence."""

    # ✅ Retrieve session language or set default
    if session_id not in session_data:
        set_session_language(session_id, "English")

    if language:
        set_session_language(session_id, language)
    else:
        language = session_data[session_id]["language"]  

    print(f"🟢 Current session language: {session_data[session_id]['language']}")  # ✅ Debugging

    # ✅ Store user message in Supabase before calling chatbot
    for msg in messages:
        save_message_to_supabase(session_id, "user", msg.content)

    # ✅ Retrieve stored messages from session history
    existing_messages = session_data[session_id]["history"].messages

    # ✅ Apply trimming BEFORE chatbot invocation
    trimmed_messages = trimmer.invoke(existing_messages + messages)

    # ✅ Replace session history with trimmed messages
    session_data[session_id]["history"].messages = trimmed_messages  

    # ✅ Invoke chatbot with proper language enforcement
    response = with_message_history.invoke(
        {
            "messages": trimmed_messages,
            "language": language  # ✅ Ensure language is included
        },
        config={"configurable": {"session_id": session_id}}
    )

    print(f"🟢 Chatbot response language: {language}")  # ✅ Debugging
    print(f"🟢 Chatbot raw response: {response.content}")  # ✅ Debugging

    # ✅ Store chatbot response in Supabase
    save_message_to_supabase(session_id, "assistant", response.content)

    # ✅ Store trimmed messages back to session to avoid memory overflow
    session_data[session_id]["history"].messages = trimmed_messages  

    return response


In [252]:
#now doing memory management to remeber the content inside context window
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# ✅ Token counter function (basic approximation)
def token_counter(messages):
    """Simple token counter using word count approximation."""
    return sum(len(msg.content.split()) for msg in messages)

trimmer = trim_messages(
    max_tokens=5000,
    strategy = "last",
    #include_last = True,
    include_system = True,
    allow_partial = False,
    start_on = "human",
    token_counter=token_counter,
)

In [253]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. The user may ask in any language, "
            "but you must always respond in {language}. **Never reply in a different language.**"
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [254]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)  #Trim messages before passing
    | prompt
    | model
)

In [255]:
from langchain_core.runnables.history import RunnableWithMessageHistory
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")



In [256]:
config_1 = {"configurable": {"session_id": "chat_1"}} #testing with creating a session id

In [257]:
response = invoke_with_language(
    session_id="chat_1",
    messages=[
        HumanMessage(content="Hi, I am Bob."),
        SystemMessage(content="Hi Bob! How can I assist you today?"),  # ✅ Should be ignored
        HumanMessage(content="Can you give me a cool and catchy nickname?")
    ],
    language="English"
)

print("Chatbot Response:", response.content)

🟢 Current session language: English
Supabase API Response: data=[{'id': '897c71de-52e4-4e22-a313-f227e7313ec3', 'session_id': 'chat_1', 'user_id': 'NULL', 'message': 'Hi, I am Bob.', 'role': 'user', 'timestamp': '2025-02-21T01:37:49.58461'}] count=None
Supabase API Response: data=[{'id': '048d42f7-562e-4018-b52f-ed523c4949e9', 'session_id': 'chat_1', 'user_id': 'NULL', 'message': 'Hi Bob! How can I assist you today?', 'role': 'user', 'timestamp': '2025-02-21T01:37:49.643647'}] count=None
Supabase API Response: data=[{'id': 'ac37f51d-56de-4948-87f1-5fe0ee41b24c', 'session_id': 'chat_1', 'user_id': 'NULL', 'message': 'Can you give me a cool and catchy nickname?', 'role': 'user', 'timestamp': '2025-02-21T01:37:49.700054'}] count=None
🟢 Chatbot response language: English
🟢 Chatbot raw response: Of course, Bob! How about "Bobby Blaze"?
Supabase API Response: data=[{'id': '90ccca96-26d1-4096-8d46-cd3327aa9f4a', 'session_id': 'chat_1', 'user_id': 'NULL', 'message': 'Of course, Bob! How about 

In [258]:
print("Response in English:", response.content)

Response in English: Of course, Bob! How about "Bobby Blaze"?


In [259]:
response = invoke_with_language(
    session_id="chat_1",
    messages=[HumanMessage(content="What's my name?")],
    language="Hindi"
)

print("Response in Hindi:", response.content)

🟢 Current session language: Hindi
Supabase API Response: data=[{'id': '2d444476-45c8-413a-8666-45c0e27cf22f', 'session_id': 'chat_1', 'user_id': 'NULL', 'message': "What's my name?", 'role': 'user', 'timestamp': '2025-02-21T01:37:53.066955'}] count=None
🟢 Chatbot response language: Hindi
🟢 Chatbot raw response: आपका नाम बॉब है।
Supabase API Response: data=[{'id': 'a044b011-2efa-4626-ae91-f0be856f588e', 'session_id': 'chat_1', 'user_id': 'NULL', 'message': 'आपका नाम बॉब है।', 'role': 'assistant', 'timestamp': '2025-02-21T01:37:57.370237'}] count=None
Response in Hindi: आपका नाम बॉब है।


In [260]:
for i in range(20):  # ✅ Send multiple messages to trigger trimming
    response = invoke_with_language(
        session_id="chat_1",
        messages=[HumanMessage(content=f"Message {i+1}: This is a long conversation testing memory management.")],
        language="English"
    )

response = invoke_with_language(
    session_id="chat_1",
    messages=[HumanMessage(content="What is my first message?")],
    language="English"
)

print("Final Response After Trimming:", response.content)  # ✅ Bot should not recall old messages


🟢 Current session language: English
Supabase API Response: data=[{'id': '6bdd822d-03b2-4c48-8808-5da648e1a636', 'session_id': 'chat_1', 'user_id': 'NULL', 'message': 'Message 1: This is a long conversation testing memory management.', 'role': 'user', 'timestamp': '2025-02-21T01:37:59.361565'}] count=None
🟢 Chatbot response language: English
🟢 Chatbot raw response: I'm here to help! How can I assist you today?
Supabase API Response: data=[{'id': '8566817e-2bb0-4932-840c-48e99c60c7f0', 'session_id': 'chat_1', 'user_id': 'NULL', 'message': "I'm here to help! How can I assist you today?", 'role': 'assistant', 'timestamp': '2025-02-21T01:38:01.220074'}] count=None
🟢 Current session language: English
Supabase API Response: data=[{'id': '4f87b2d7-0647-496c-8f73-de9e0d70f42d', 'session_id': 'chat_1', 'user_id': 'NULL', 'message': 'Message 2: This is a long conversation testing memory management.', 'role': 'user', 'timestamp': '2025-02-21T01:38:01.274985'}] count=None
🟢 Chatbot response languag